In [1]:
from login import Login
import requests
import json
from typing import List
import time
import random

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/ 537.36 Edg/107.0.1418.62'
}


In [3]:

# Login

login = Login()
url, token = login.apply_QR_code()
login.generate_QR_code(url)
input('已扫码')
bilibili_session = requests.Session()
code = login.do_web_login(bilibili_session, token)
if code == 0:
    print('登录成功')
elif code == 86038:
    print('二维码失效')
elif code == 86090 or code == 86101:
    print('请扫码并确认')
else:
    print(f'错误，code:{code}')


█████████████████████████████████████████████████████
█████████████████████████████████████████████████████
████ ▄▄▄▄▄ ██▄▄█ ▀██ ▄█ ▄█ ██▀ █ ▄  ▀▀█▀▄█ ▄▄▄▄▄ ████
████ █   █ ██ █▄ ▄▄▄▄▀▄▀▀  ▄  ▀██▀▄ ▄▀█ ▀█ █   █ ████
████ █▄▄▄█ █▀█▀███▀█  ▀▀ ▄▄▄ █▄ ▀█ ▀ ▄▄▄▄█ █▄▄▄█ ████
████▄▄▄▄▄▄▄█▄▀▄▀ ▀▄▀▄▀ ▀ █▄█ ▀ ▀▄█▄█ ▀ █▄█▄▄▄▄▄▄▄████
████▄▀█▄█▄▄▀▄▄ █ ▄ ▀▀▀█▄  ▄ ▄██▄▄█▄▄▀█▄▄█ █▄▀███▀████
████▀ ▀▄▄ ▄▄█▀█▄▄ ▄██ █▀  █▀▄▄█▀ ▄█▄█▀▄▄█▀▀█▀▄▄▀ ████
████▄█ █ █▄▄██ ▀ █  █▀ ▄▄▄█▄▀██ ▄ ▄▄ ▄▀▄██▄▀▄██▀▀████
████ ▄  ▀▀▄▄█ ▄ ▀█ ▄▀▀ █▄█▀▄▄▀▄  ▀█▀▀▄▀▀▄█▄  ▄▄▀▀████
██████ █ ▄▄█   ▄▄▀▄▀ ▄ ▀▀▀▀▄ █▀▄█▀█▀ ▄▄ █▄▀████▀▄████
████  █▄█▀▄▄▀ ▀▄▀▀▄█ ▄ ▀█  ▄ ██▀█▄▀█ █▀▄▀ ██▄▀▄ ▄████
████ ▄ ▀ ▄▄▄ ▀█ ██▀▄█▄ █ ▄▄▄ █▄▄ ██▄█▀██ ▄▄▄ ███ ████
████▄▄▀█ █▄█    █▄ ▄ ▀▄█ █▄█  █ ▀█▀ █ ▄█ █▄█ ▀▄  ████
████▄▀▀ ▄ ▄▄ ▄██ ██  █▄▀  ▄ ▄ ▄ ▄ ▄  ▄ ▄▄▄▄▄ ▄▀█▀████
█████▀███▄▄▀ ▄▄▄ █ ▄██▄█▄ ▀▄██ ▀ ▀▀▄  █▄  ▀▀▄█▄ █████
████ ▀█ █ ▄█▄▀▄ █▀▄▄▀█▄ ▄█▀▄ █▀█▀ ▄█▄▄▄▀▄ █▄▄▀██▄████
████ ▄▄█ █▄██  █▄  ▄ ▄█  ▀▀ █▄█▄▄██ ▀▄▀█ █ ▀  ▄▀ ████
████▄▀ █▄█▄█▀ ▄█▀█▄▄█▀▀▄▀█ █

In [4]:
# Get mid

def get_mid(session: requests.Session) -> int:
    res = bilibili_session.get(
        url='https://api.bilibili.com/nav', headers=headers)
    res = json.loads(res.text)
    data = res['data']
    return data['mid']


In [9]:
# Get the total number of followings

def get_total_follow(session: requests.Session) -> int:
    mid = get_mid(session)
    res = session.get(url='https://api.bilibili.com/x/relation/followings',
                      headers=headers,
                      params={'vmid': mid})
    data = json.loads(res.text)['data']
    return data['total']

# Get following list


def get_follow_list(session: requests.Session, page: int = 1) -> List[dict]:
    mid = get_mid(session)
    res = session.get(url='https://api.bilibili.com/x/relation/followings',
                      headers=headers,
                      params={'vmid': mid, 'pn': page, 'ps': 50})
    data = json.loads(res.text)['data']
    return data['list']


In [11]:
total = get_total_follow(bilibili_session)
print(total)
follow_list = []
for page in range(1, total // 51 + 2): #I get the data using 50, but there will be a bug if the total number is 800 or 850
    time.sleep(random.randint(1, 3))
    ups = get_follow_list(bilibili_session, page)
    print(page, len(ups))
    follow_list.extend(ups)
print(len(follow_list))


830
1 50
2 50
3 50
4 50
5 50
6 50
7 50
8 50
9 50
10 50
11 50
12 50
13 50
14 50
15 50
16 50
17 30
830


In [1]:
# Save results

with open('following.json','w') as f:
    f.write(json.dumps(follow_list))